In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('C:/Users/udipt/Desktop/Websoft/Assignment/DataScienceCaseStudy/dataSample.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16162 entries, 0 to 16161
Data columns (total 11 columns):
Unnamed: 0               16162 non-null int64
ID                       16162 non-null int64
Type                     16162 non-null object
event_timestamp          16162 non-null int64
event_label              16162 non-null object
log_timestamp            16162 non-null int64
log_reading_1            15848 non-null float64
log_reading_2            15883 non-null object
log_reading_3            15843 non-null float64
log_reading_4            16162 non-null int64
warranty_service_flag    1389 non-null object
dtypes: float64(2), int64(5), object(4)
memory usage: 1.4+ MB


# Preprocessing

In [3]:
df=pd.get_dummies(df)
df=df.rename(columns={'Unnamed: 0':'sl_number'})
df.columns

Index(['sl_number', 'ID', 'event_timestamp', 'log_timestamp', 'log_reading_1',
       'log_reading_3', 'log_reading_4', 'Type_TypeA', 'Type_TypeB',
       'event_label_activate', 'event_label_break', 'log_reading_2_a',
       'log_reading_2_b', 'log_reading_2_c', 'warranty_service_flag_n',
       'warranty_service_flag_y'],
      dtype='object')

In [4]:
df=df.drop(columns=['event_label_activate','warranty_service_flag_n'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16162 entries, 0 to 16161
Data columns (total 14 columns):
sl_number                  16162 non-null int64
ID                         16162 non-null int64
event_timestamp            16162 non-null int64
log_timestamp              16162 non-null int64
log_reading_1              15848 non-null float64
log_reading_3              15843 non-null float64
log_reading_4              16162 non-null int64
Type_TypeA                 16162 non-null uint8
Type_TypeB                 16162 non-null uint8
event_label_break          16162 non-null uint8
log_reading_2_a            16162 non-null uint8
log_reading_2_b            16162 non-null uint8
log_reading_2_c            16162 non-null uint8
warranty_service_flag_y    16162 non-null uint8
dtypes: float64(2), int64(5), uint8(7)
memory usage: 994.5 KB


In [5]:
df['log_reading_1'].fillna(df['log_reading_1'].median(),inplace=True)
df['log_reading_3'].fillna(df['log_reading_3'].median(),inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16162 entries, 0 to 16161
Data columns (total 14 columns):
sl_number                  16162 non-null int64
ID                         16162 non-null int64
event_timestamp            16162 non-null int64
log_timestamp              16162 non-null int64
log_reading_1              16162 non-null float64
log_reading_3              16162 non-null float64
log_reading_4              16162 non-null int64
Type_TypeA                 16162 non-null uint8
Type_TypeB                 16162 non-null uint8
event_label_break          16162 non-null uint8
log_reading_2_a            16162 non-null uint8
log_reading_2_b            16162 non-null uint8
log_reading_2_c            16162 non-null uint8
warranty_service_flag_y    16162 non-null uint8
dtypes: float64(2), int64(5), uint8(7)
memory usage: 994.5 KB


# Train test split

In [6]:
x=df.drop(columns=['event_label_break'],axis=1)
y=df['event_label_break']
print(f'The data has {y.mean()*100}% of positive points')

The data has 34.14800148496473% of positive points


In [7]:
from sklearn.model_selection import train_test_split
x_tr,x_te,y_tr,y_te=train_test_split(x,y,stratify=y,test_size=.33,random_state=10)
print(x_tr.shape,x_te.shape,y_tr.shape,y_te.shape)
print(f'Training set has {y_tr.mean()*100}% positive points and test sets {y_te.mean()*100}% of positive points')

(10828, 13) (5334, 13) (10828,) (5334,)
Training set has 34.152198005171776% positive points and test sets 34.13948256467941% of positive points


# Decision tree

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
from sklearn.model_selection import GridSearchCV

In [45]:
dt=DecisionTreeClassifier(random_state=10)
dt.fit(x_tr,y_tr)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=10, splitter='best')

In [46]:
pgd={'max_depth':range(3,10),'min_samples_split':range(2,6),'max_features':['sqrt','log2',None]}
dt_cv=GridSearchCV(dt,pgd,cv=5,scoring='roc_auc',n_jobs=-1)

In [47]:
dt_cv.fit(x_tr,y_tr)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=10,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': range(3, 10),
                         'max_features': ['sqrt', 'log2', None],
                       

In [48]:
print(f'Best parameters are {dt_cv.best_params_}, and the best score is {dt_cv.best_score_}')

Best parameters are {'max_depth': 5, 'max_features': None, 'min_samples_split': 2}, and the best score is 0.8141247911428451


In [49]:
dt_best=dt_cv.best_estimator_

In [50]:
yd1_prob=dt.predict_proba(x_te)
yd1_pred=dt.predict(x_te)
print(f'Decision tree without grid search predicted {yd1_pred.mean()*100}% points as positive in test data')

Decision tree without grid search predicted 34.15823022122235% points as positive in test data


In [51]:
ydt_prob=dt_best.predict_proba(x_te)
ydt_pred=dt_best.predict(x_te)
print(f'Decision tree predicted {ydt_pred.mean()*100}% points as positive in test data')

Decision tree predicted 49.08136482939632% points as positive in test data


# Random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [81]:
rf=RandomForestClassifier(n_jobs=-1, random_state=10)
rf.fit(x_tr,y_tr)

C:\Users\udipt\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [91]:
pgr={'max_features':['sqrt',None], 'n_estimators':range(50,351,50)}
rf_cv=GridSearchCV(rf,pgr,cv=5,scoring='roc_auc',n_jobs=-1)
rf_cv.fit(x_tr,y_tr)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=-1,
                                              oob_score=False, random_state=10,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_feat

In [92]:
print(f'Best parameters after grid search are {rf_cv.best_params_} and best score is {rf_cv.best_score_}')

Best parameters after grid search are {'max_features': None, 'n_estimators': 100} and best score is 0.7520188902755699


In [93]:
rf_best=rf_cv.best_estimator_

In [94]:
yr1_prob=rf.predict_proba(x_te)
yr1_pred=rf.predict(x_te)
print(f'Random forest without grid search predicted {yr1_pred.mean()*100}% points as positive in test data')

Random forest without grid search predicted 29.977502812148483% points as positive in test data


In [189]:
yrf_prob=rf_best.predict_proba(x_te)
yrf_pred=rf_best.predict(x_te)
print(f'Random forest with grid search predicted {yrf_pred.mean()*100}% points as positive in test data')

Random forest with grid search predicted 35.470566179227596% points as positive in test data


# XGboost

In [68]:
from xgboost import XGBClassifier

In [180]:
xg=XGBClassifier(max_depth=2,n_jobs=-1)
xg.fit(x_tr,y_tr)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [182]:
pgx={'learning_rate':[.3,.35,.45], 'scale_pos_weight':[.51,.6,.75]}
xg_cv=GridSearchCV(xg,pgx,cv=5,scoring='roc_auc',n_jobs=-1)
xg_cv.fit(x_tr,y_tr)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=2,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_esti...
                                     num_parallel_tree=1,
                                     objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
         

In [183]:
print(f'Best parameters after grid search are {xg_cv.best_params_} and best score is {xg_cv.best_score_}')

Best parameters after grid search are {'learning_rate': 0.3, 'scale_pos_weight': 0.75} and best score is 0.8264928767728792


In [184]:
xg_best=xg_cv.best_estimator_

In [181]:
yx1_prob=xg.predict_proba(x_te)
yx1_pred=xg.predict(x_te)
print(f'XGboost without grid search predicted {yx1_pred.mean()*100}% points as positive in test data')

XGboost without grid search predicted 42.1822272215973% points as positive in test data


In [187]:
yxg_prob=xg_best.predict_proba(x_te)
yxg_pred=xg_best.predict(x_te)
print(f'XGboost with grid search predicted {yxg_pred.mean()*100}% points as positive in test data')

XGboost with grid search predicted 30.2212223472066% points as positive in test data
